## Reading File Data to DataFrame

In [ ]:
#import module
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#create session in order to be capable of accessing all Spark API
spark = SparkSession \
    .builder \
    .appName("Introdution to Spark DataFrame") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#define data schema for file we want to read
purchaseSchema = StructType([
    StructField("Date", DateType(), True),
    StructField("Time", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Item", StringType(), True),
    StructField("Total", FloatType(), True),
    StructField("Payment", StringType(), True),
])    

# read csv file with our defined schema into Spark DataFrame, and use "tab" delimiter
purchaseDataframe = spark.read.csv(
    "random_sales_data.csv", 
    header=True, schema=purchaseSchema, sep=",")  # Change sep="\t" to sep=","
purchaseDataframe.show(3)

## Counting number of rows, printing dataframe schema and printing statistic of our data

In [5]:
#count number of rows of our dataFrame
num_rows = purchaseDataframe.count()
print("number of rows: ", num_rows)
#show our dataFrame schema
purchaseDataframe.printSchema()
#show statistic of the data we want
purchaseDataframe.describe('Total').show()

number of rows:  100
root
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Total: float (nullable = true)
 |-- Payment: string (nullable = true)

+-------+-----------------+
|summary|            Total|
+-------+-----------------+
|  count|              100|
|   mean|982.8642013549804|
| stddev| 575.460867997704|
|    min|            71.38|
|    max|          1999.51|
+-------+-----------------+



## Creating a new dataFrame from a subset of our existing dataFrame

In [6]:
#create new dataFrame from "City" and "Total" columns
newDataframe = purchaseDataframe.select(purchaseDataframe['City'], 
                                              purchaseDataframe['Total'])
newDataframe.show(3); #menampilkan 3 baris DataFrame baru kita
newDataframe.printSchema() #print skema dari DataFrame baru kita

+--------+-------+
|    City|  Total|
+--------+-------+
|New York|1999.51|
|New York|1690.17|
|New York|  78.16|
+--------+-------+
only showing top 3 rows

root
 |-- City: string (nullable = true)
 |-- Total: float (nullable = true)



## Adding a constant value to every row data in certain column

In [7]:
#adding constant value of 10 to every row data in "Total" column
purchaseDataframe.select(purchaseDataframe['City'],
                         purchaseDataframe['Total']+10).show(3)

+--------+------------+
|    City|(Total + 10)|
+--------+------------+
|New York|     2009.51|
|New York|     1700.17|
|New York|       88.16|
+--------+------------+
only showing top 3 rows



## Filtering dataFrame using our defined condition

In [8]:
#filter only row data whose "Total" column value > 200
purchaseDataframe.filter(purchaseDataframe['Total'] > 200).show(3)

+----------+--------+-----------+--------+-------+-------+
|      Date|    Time|       City|    Item|  Total|Payment|
+----------+--------+-----------+--------+-------+-------+
|2023-02-07|02:47:33|   New York|   Phone|1999.51| PayPal|
|2023-06-21|10:46:46|   New York|Keyboard|1690.17| PayPal|
|2023-05-30|08:40:31|Los Angeles|   Mouse|1641.63|   Cash|
+----------+--------+-----------+--------+-------+-------+
only showing top 3 rows



## Sorting dataFrame by certain column

In [9]:
sortedByCity = purchaseDataframe.orderBy('City').show(4)

+----------+--------+-------+--------+-------+----------+
|      Date|    Time|   City|    Item|  Total|   Payment|
+----------+--------+-------+--------+-------+----------+
|2023-04-04|07:36:51|Chicago|Keyboard|1870.75|Debit Card|
|2023-02-09|14:17:55|Chicago|   Phone| 293.42|Debit Card|
|2023-12-07|13:07:18|Chicago|  Laptop| 816.82|Debit Card|
|2023-06-15|00:16:21|Chicago|   Phone|1963.72|      Cash|
+----------+--------+-------+--------+-------+----------+
only showing top 4 rows



## Calculating number of transactions in each city

In [10]:
numTransactionEachCity = purchaseDataframe.groupBy("City").count()
numTransactionEachCity.show(5)

+-----------+-----+
|       City|count|
+-----------+-----+
|    Phoenix|   22|
|Los Angeles|   20|
|    Chicago|   13|
|    Houston|   21|
|   New York|   24|
+-----------+-----+



## Indexing and Accessing DataFrame
Since Spark dataFrame is distributed into clusters, we cannot access it by [row,column] as in pandas dataFrame for example. There is an alternative way to do that by creating new column with "incremental ID". Then, we can access by row using ".filter()" function to our "incremental ID" column. 

In [11]:
#import monotonically_increasing_id
from pyspark.sql.functions import monotonically_increasing_id

newPurchasedDataframe = purchaseDataframe.withColumn(
    "index", monotonically_increasing_id())
newPurchasedDataframe.show(7)
row2Till4 = newPurchasedDataframe.filter((newPurchasedDataframe['index']>=2) &
                                         (newPurchasedDataframe['index']<=4))
row2Till4.show()

+----------+--------+-----------+--------+-------+-----------+-----+
|      Date|    Time|       City|    Item|  Total|    Payment|index|
+----------+--------+-----------+--------+-------+-----------+-----+
|2023-02-07|02:47:33|   New York|   Phone|1999.51|     PayPal|    0|
|2023-06-21|10:46:46|   New York|Keyboard|1690.17|     PayPal|    1|
|2023-07-03|09:41:10|   New York| Monitor|  78.16| Debit Card|    2|
|2023-05-30|08:40:31|Los Angeles|   Mouse|1641.63|       Cash|    3|
|2023-02-20|10:38:44|   New York|   Phone|1499.96|Credit Card|    4|
|2023-08-15|13:28:17|   New York|Keyboard| 775.31|Credit Card|    5|
|2023-09-02|08:17:51|   New York|  Tablet|1387.94|Credit Card|    6|
+----------+--------+-----------+--------+-------+-----------+-----+
only showing top 7 rows

+----------+--------+-----------+-------+-------+-----------+-----+
|      Date|    Time|       City|   Item|  Total|    Payment|index|
+----------+--------+-----------+-------+-------+-----------+-----+
|2023-07-03|

Then, to access it by row and column, use ".select()" function we ever used above before.

In [12]:
dataRow2ColumnTotal = newPurchasedDataframe.filter(newPurchasedDataframe['index']==2).select('Total')
dataRow2ColumnTotal.show()

+-----+
|Total|
+-----+
|78.16|
+-----+



# Using SQL query in dataFrame

Create a new dataFrame from the subset of our existing dataFrame

In [13]:
#we need to make sql temporary view for our dataFrame
purchaseDataframe.createOrReplaceTempView("purchaseSql")

#select "Total" dan "Payment" column from our sql temporary view
anotherNewDataframe = spark.sql("SELECT Total, Payment FROM purchaseSql")
anotherNewDataframe.show(3)

+-------+----------+
|  Total|   Payment|
+-------+----------+
|1999.51|    PayPal|
|1690.17|    PayPal|
|  78.16|Debit Card|
+-------+----------+
only showing top 3 rows



Sorting data by certain column

In [14]:
#sorting data by "City" column alphabetically
orderByCity = spark.sql("SELECT * FROM purchaseSql ORDER BY City")
orderByCity.show(5)

+----------+--------+-------+------+-------+-----------+
|      Date|    Time|   City|  Item|  Total|    Payment|
+----------+--------+-------+------+-------+-----------+
|2023-12-07|13:07:18|Chicago|Laptop| 816.82| Debit Card|
|2023-04-09|20:24:19|Chicago| Mouse| 918.36|Credit Card|
|2023-06-14|09:32:38|Chicago| Phone|1250.92| Debit Card|
|2023-06-15|00:16:21|Chicago| Phone|1963.72|       Cash|
|2023-01-16|14:33:11|Chicago| Mouse|1284.71|       Cash|
+----------+--------+-------+------+-------+-----------+
only showing top 5 rows



The last one, we wil show how to filter data with "Total" column value > 200, and sort them by "Payment" column data.

In [16]:
#filter nilai kolom Total>50 dan urutkan berdasarkan cara pembayaran
filterAndSortWithSQL = spark.sql("SELECT * FROM purchaseSql WHERE Total>200 ORDER BY Payment")
filterAndSortWithSQL.show(4)

+----------+--------+-------+--------+-------+-------+
|      Date|    Time|   City|    Item|  Total|Payment|
+----------+--------+-------+--------+-------+-------+
|2023-06-15|00:16:21|Chicago|   Phone|1963.72|   Cash|
|2023-11-16|03:32:45|Phoenix|Keyboard| 1684.1|   Cash|
|2023-12-04|22:45:48|Phoenix|  Laptop| 281.72|   Cash|
|2023-06-16|03:37:18|Chicago| Monitor| 451.94|   Cash|
+----------+--------+-------+--------+-------+-------+
only showing top 4 rows

